# Few Installations

In [ ]:
!pip install --quiet ipython-autotime

In [ ]:
!pip install -U openai-whisper

# Convert text to speech

In [ ]:
!pip install edge-tts asyncio

In [ ]:
import os
import asyncio
import edge_tts
import nest_asyncio
from google.colab import files

nest_asyncio.apply()

VOICE = "en-US-JennyNeural"
OUTPUT_FILE = "output.mp3"

async def convert_text_to_audio(text):
    communicate = edge_tts.Communicate(text.strip(), VOICE)
    await communicate.save(OUTPUT_FILE)
    print(f"Audio saved as {OUTPUT_FILE}")

text = "Why did the dog sit in the shade? Because he didn’t want to become a hot dog!"
await convert_text_to_audio(text)

# Install whisper and convert mp3 to subtitle

In [ ]:
import whisper

model = whisper.load_model("medium")
result = model.transcribe("output.mp3",word_timestamps=True)
print(result)

print (result['text'])

for each in result['segments']:
  print (each)

wordlevel_info = []

for each in result['segments']:
  words = each['words']
  for word in words:
    # print (word['word'], "  ",word['start']," - ",word['end'])
    wordlevel_info.append({'word':word['word'].strip(),'start':word['start'],'end':word['end']})

In [ ]:
import json
with open('data.json', 'w') as f:
    json.dump(wordlevel_info, f,indent=4)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Downloading Minecraft video

In [ ]:
!pip install yt-dlp

In [ ]:
import yt_dlp

save_path = '/content/drive/MyDrive/minecraft.mp4'

ydl_opts = {
    'outtmpl': save_path,
    'format': 'bestvideo+bestaudio/best',
}

url = 'https://www.youtube.com/watch?v=xKRNDalWE-E'

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

# Moviepy

In [ ]:
!pip install moviepy

In [ ]:
from moviepy.editor import AudioFileClip

audio = AudioFileClip("output.mp3")
audio_duration = audio.duration
print(f"MP3 Duration: {audio_duration} seconds")

In [ ]:
import random
import os

video = '/content/drive/MyDrive/minecraft.mkv'

In [ ]:
from moviepy.editor import VideoFileClip

video = VideoFileClip(video)

duration = video.duration

print(f"The duration of the video is: {duration} seconds")

In [ ]:
import random

if duration < audio_duration:
    raise ValueError("The video is shorter than the required trim length.")

random_start = random.uniform(0, duration - audio_duration)


trimmed_video = video.subclip(random_start, random_start + audio_duration)

trimmed_video_path = "trimmed_video.mp4"
trimmed_video.write_videofile(trimmed_video_path, codec="libx264")

video.close()
trimmed_video.close()

In [ ]:
import json

with open('data.json', 'r') as f:
    subtitles_data = json.load(f)

print(subtitles_data)

# Add subtitle to video

In [ ]:
!apt-get install -y imagemagick

In [ ]:
!echo "<policys>" > /etc/ImageMagick-6/policy.xml
!echo "<policy domain='coder' rights='read|write' pattern='*' />" >> /etc/ImageMagick-6/policy.xml
!echo "</policys>" >> /etc/ImageMagick-6/policy.xml

In [ ]:
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip

video = VideoFileClip("trimmed_video.mp4")

subtitles_clips = []

for subtitle in subtitles_data:
    text = TextClip(subtitle['word'], font="Poppins-Regular.ttf", fontsize=100, color='white')
    text = text.set_start(subtitle['start']).set_duration(subtitle['end'] - subtitle['start'])
    text = text.set_position(('center', 'center'))
    subtitles_clips.append(text)

final_video = CompositeVideoClip([video] + subtitles_clips)
final_video.write_videofile("minecraft_with_subtitles.mp4", fps=video.fps)


# Finally merge video and audio

In [ ]:
import yt_dlp

save_path = '/content/drive/MyDrive/background.%(ext)s'

ydl_opts = {
    'outtmpl': save_path,
    'format': 'bestaudio/best',
    'postprocessors': [
        {  # Extract audio using ffmpeg
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }
    ],
}

url = 'https://www.youtube.com/watch?v=vi84zA6iC7w'

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip

video = VideoFileClip("minecraft_with_subtitles.mp4")

audio = AudioFileClip("output.mp3")

final_video = video.set_audio(audio)

final_video.write_videofile("output.mp4", fps=video.fps)

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip

video_clip = VideoFileClip("output.mp4")
original_audio = video_clip.audio

background_audio = AudioFileClip("/content/drive/MyDrive/background.mp3").volumex(0.11)

background_audio = background_audio.set_duration(video_clip.duration)

combined_audio = CompositeAudioClip([original_audio, background_audio])

final_clip = video_clip.set_audio(combined_audio)

final_clip.write_videofile("output_with_music.mp4", codec="libx264", audio_codec="aac")

In [ ]:
from google.colab import files
files.download("output_with_music.mp4")